In [1]:
from utils.tobj import import_tobj
import meshio
import numpy as np

V, T = import_tobj("data/bunny.tobj")
mesh = meshio.Mesh(points = V, cells = [("tetra", T)])
mesh.write("assets/bunny.msh", file_format="gmsh22")

(3670, 3) (14965, 4)


Warning: Appending zeros to replace the missing physical tag data.

Warning: Appending zeros to replace the missing geometrical tag data.

In [3]:
obj_in = meshio.read("assets/bunny.obj")
off_out = meshio.write("assets/bunny.off", obj_in, file_format="off")